In [2]:
import re
import pandas as pd

In [3]:
import ast
import json

In [29]:
alphabets= "([A-Za-z])"
prefixes = "(Mr|St|Mrs|Ms|Dr)[.]"
suffixes = "(Inc|Ltd|Jr|Sr|Co)"
starters = "(Mr|Mrs|Ms|Dr|He\s|She\s|It\s|They\s|Their\s|Our\s|We\s|But\s|However\s|That\s|This\s|Wherever)"
acronyms = "([A-Z][.][A-Z][.](?:[A-Z][.])?)"
websites = "[.](com|net|org|io|gov)"

def split_into_sentences(text):
    text = " " + text + "  "
    text = text.replace("\n"," ")
    text = re.sub(prefixes,"\\1<prd>",text)
    text = re.sub(websites,"<prd>\\1",text)
    if "Ph.D" in text: text = text.replace("Ph.D.","Ph<prd>D<prd>")
    text = re.sub("\s" + alphabets + "[.] "," \\1<prd> ",text)
    text = re.sub(acronyms+" "+starters,"\\1<stop> \\2",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>\\3<prd>",text)
    text = re.sub(alphabets + "[.]" + alphabets + "[.]","\\1<prd>\\2<prd>",text)
    text = re.sub(" "+suffixes+"[.] "+starters," \\1<stop> \\2",text)
    text = re.sub(" "+suffixes+"[.]"," \\1<prd>",text)
    text = re.sub(" " + alphabets + "[.]"," \\1<prd>",text)
    if "”" in text: text = text.replace(".”","”.")
    if "\"" in text: text = text.replace(".\"","\".")
    if "!" in text: text = text.replace("!\"","\"!")
    if "?" in text: text = text.replace("?\"","\"?")
    text = text.replace(".",".<stop>")
    text = text.replace("?","?<stop>")
    text = text.replace("!","!<stop>")
    text = text.replace("<prd>",".")
    sentences = text.split("<stop>")
    sentences = sentences[:-1]
    sentences = [s.strip() for s in sentences]
    return sentences



In [3]:
#text=['Dr Nelson and her staff are wonderful! They are always on time and very caring!', 'Dr. Nelson was our family pediatrician for over 20 years when I was a child. I am a first time mom and my children will not go to any other pediatrician. Dr. Nelson is thoughtful, knowledge, and wonderful with both the children and parents. She is the best pediatrician.', 'Dr Nelson is amazing. One thing that I\'ve noticed sets her apart is she truly knows and cares for all of her patients. You are not just another name scheduled for the day. Martine is an extremely confident, easy going, intelligent, and compassionate provider. In my opinion you are hard pressed to find that combo many places.', 'Been going here for 21 years. BEST DOCTOR AROUND!! I love Doctor Nelson :)', 'I have a five year old and a three year old and they have been going to Dr. Nelson since they were born. We all love Dr. Nelson. She is knowledgeable, kind and thorough. We have had several issues over the years and Dr. Nelson and her staff have always been very helpful!', 'Dr Nelson is a blessing! She is always very prompt, attentive and most importantly, available whenever you need her! Her staff is exceptional too.', 'I saw Dr. Nelson for 22 years and she is amazing! Always very helpful and attentive, a wonderful Dr.', 'Dr. Nelson is the ultimate best doctor. Our family has been with her for about 21 years and we trust her unconditionally. She always takes the time to explain everything. When our kids were ill, we were always able to get in to see her within a couple of hours - amazing! We feel very fortunate to have found her for our pediatrician. Thank you Dr. Nelson!!', "I have a three year old and a new born. Dr. Nelson has been great with both of them. When my daughter was having some problems, Dr. Nelson called us daily to check in on her. I was a very tense first time mom and Dr. Nelson's office always answered all my questions and they never made me feel bad for calling them all the time!", "I felt that this doctor was much too busy. Each time I saw her she didn't seem to remember the details of my situation. I usually had to wait at least an hour every appointment. Once I even waited 3 hours! In addition she didn't explain things very thoroughly. She does seem to be a friendly person but overall I felt insignificant to this doctor and will not be going to her for any future pregnancies.", "I've been taking my kids to Dr. Nelson since my 5 year old was born. She's great! And her staff is wonderful too!", 'Dr. Nelson has been my "19" year olds doctor since birth! We were her first patient and would follow her anywhere! She is a blessing!!!', 'Doctor Nelson is wonderful! She has a ready ear, returns calls personally, answers questions, quells fears and educated her patients.', 'Very knowledgeable, thorough, easy to talk to, and compassionate. I went through a scary problem, and she was very supportive and thorough in testing, explaining treatment and options. Highly recommended.']
#sen=split_into_sentences(str(text))

In [4]:
#t2=str(text)
#sent=re.split(r'[\.\?!][\'"\)\]]*',t2)
#print(type(sent))

In [24]:
#hgrd=pd.read_excel(r'C:\Users\rramn\Documents\Healthcare_analytics\physician_reviews_mining\Reviews\healthgrades_reviews.xlsx')
ratemd=pd.read_csv(r'/Users/sujaikarunakaran/Desktop/Assignments/Spring 2020/Ranga/Final Project/Final/Healthgrades/All_Healthgrades_v3.csv')
#ratemd = pd.read_csv(r'/Users/sujaikarunakaran/Desktop/Assignments/Spring 2020/Ranga/Final Project/Final/RateMds/complete_data_ratemds.csv')
ratemd.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 840 entries, 0 to 839
Data columns (total 8 columns):
DoctorName             840 non-null object
Age                    839 non-null float64
Affiliated Hospital    839 non-null object
Gender                 840 non-null object
Education              834 non-null object
Experience             840 non-null int64
Rating                 840 non-null float64
Reviews                840 non-null object
dtypes: float64(2), int64(1), object(5)
memory usage: 52.6+ KB


In [102]:
from nltk import tokenize

In [260]:
type(ratemd["Doctor_reviews"])

pandas.core.series.Series

In [262]:
type(ratemd["Reviews"])

pandas.core.series.Series

In [214]:
ratemd["Single_review"] = ratemd["Doctor_reviews"].to_list()

In [309]:
#senthg=[]
sentrm=[]
doc_names = []
star_rating = []
gender = []
hos_affil = []
#nr,nc=hgrd.shape
nr1,nc1=ratemd.shape
#hgrd.info()
#for i in range(nr):
#    t=hgrd.iloc[i,6]
#    sent=re.split(r'[\.\?!][\'"\)\]]*',t)
#    senthg.append(sent)

for j in range(nr1):
    name = ratemd.iloc[j,0]
    #rating = ratemd.iloc[j,3]
    #gend = ratemd.iloc[j,1]
    #ha = ratemd.iloc[j,4]
    t1=ratemd.iloc[j,7]
    t1 = t1.strip("][").split(",")
    #sent1 = tokenize.sent_tokenize(t1)
    #sent1 = split_into_sentences(t1)
    #sent1=re.split(r'[\.\?!][\'"\)\]]*|,|and|but',t1)
    #sent1=re.split(r"'|.|!|,|\"|?|and|but",t1)
    #sent1=re.split(r'\',"', t1)
    #for x in sent1:
    for x in range(len(t1)):
        x_sent = t1[x]
        sentrm.append(x_sent)
        doc_names.append(name)
    #star_rating.append(rating)
    #gender.append(gend)
    #hos_affil.append(ha)
        

In [310]:
len(sentrm)

10424

In [311]:
de = {"Doc_names":doc_names, "Review_phrases":sentrm}

In [312]:
df = pd.DataFrame(de)

In [313]:
df.shape

(10424, 2)

In [111]:
#hgrd['Review_sentences']=senthg
ratemd['review_phrases']=sentrm

ValueError: Length of values does not match length of index

In [169]:
#hgrd.to_excel(r'C:\Users\rramn\Documents\Healthcare_analytics\physician_reviews_mining\Reviews\healthgrades_reviews_sentences.xlsx',header=True)
ratemd.to_excel(r'/Users/sujaikarunakaran/Desktop/Assignments/Spring 2020/Ranga/Final Project/Ratemds - Illinois/New/complete_data_sentences_withbut.xlsx',header=False)


In [69]:
#pip install vaderSentiment
#from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
#analyser1 = SentimentIntensityAnalyzer()
#def sentiment_analyzer_scores(sentence):
#    score = analyser1.polarity_scores(sentence)
    #print("{:-<40} {}".format(sentence, str(score)))
#    print(score)
#    if score['compound']>0.05:
#        return "Positive"   
#    elif score['compound']<-0.05:       
#        return "Negative"    
#    else:       
#        return "Neutral"

In [71]:
txt = "he not only loves to bring new life into the world but dr. drachler is seriously concerned about the quality of our lives"
sentiment_analyzer_scores(txt)

{'neg': 0.162, 'neu': 0.838, 'pos': 0.0, 'compound': -0.4676}


'Negative'

In [8]:
from textblob import TextBlob

In [25]:
#Function to determine the sentiment of the sentence
i=[]
j=[]
k=[]
def sentiment_analyser(text):
    analysis = TextBlob(text)
    if analysis.sentiment[0]>0:
        i.append(1)
        #return "Positive"   
        return analysis.sentiment[0]
    elif analysis.sentiment[0]<0:       
        j.append(1)
        #return "Negative"    
        return analysis.sentiment[0]
    else:       
        k.append(1)
        #return "Neutral"
        return analysis.sentiment[0]

In [177]:
nr1,nc1=df.shape

In [67]:
#function to take all the sentences, identify sentiment and provide a dictionary with key as sentence and value as
#sentiment
def sentence_sentiment(df):
    main_list = []
    nr1,nc1 = df.shape
    for j in range(nr1):
        t1=ratemd.iloc[j,2]
        sub_list = []
        for x in range(len(t1)):
            d = {}
            senti = sentiment_analyser(t1[x])
            d[t1[x]] = senti
            for f in d.copy():
                if d[f] == 'Neutral':
                    d.pop(f)
            sub_list.append(d)
        main_list.append(sub_list)
    return main_list

In [28]:
def sentence_sentiment1(df):
    main_list = []
    nr1,nc1 = df.shape
    for j in range(nr1):
        t1=df.iloc[j,7]
        senti = sentiment_analyser(t1)
        main_list.append(senti)
    return main_list

In [29]:
sentiment = sentence_sentiment1(ratemd)

In [30]:
len(sentiment)

840

In [149]:
type(sentiment[0][0])

dict

In [129]:
for x in range(len(sentiment)): 
    q = sentiment[x]
    for a in range(len(q)):
        sent = q[a]
        for f in sent.keys():
            if sent[f] == 'Neutral':
                del(sent[f])

In [131]:
for x in range(len(sentiment)): 
    q = sentiment[x]
    for a in range(len(q)):
        sent = q[a]
        for f in sent.copy():
            if not sent[f]:
                sent.pop(f)

In [156]:
filter(None, sentiment)

In [31]:
len(sentiment)
print(len(i))
print(len(j))
print(len(k))

832
254
270


In [166]:
sentiment

['Neutral',
 'Negative',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Positive',
 'Positive',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Positive',
 'Positive',
 'Positive',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Positive',
 'Neutral',
 'Positive',
 'Neutral',
 'Neutral',
 'Positive',
 'Positive',
 'Neutral',
 'Positive',
 'Neutral',
 'Neutral',
 'Neutral',
 'Positive',
 'Neutral',
 'Positive',
 'Positive',
 'Neutral',
 'Positive',
 'Neutral',
 'Positive',
 'Neutral',
 'Neutral',
 'Neutral',
 'Positive',
 'Negative',
 'Neutral',
 'Neutral',
 'Neutral',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Positive',
 'Neutral',
 'Neutral',
 'Neutral',
 'Positive',
 'Neutral',
 'Positive',
 'Neutral',
 'Neutral',
 'Neutral',
 'Positive',
 'Negative',
 'Neutral',
 'Positive',
 'Neutral',
 'Neutral',
 'Positive',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Neutral',
 'Negative',
 'Neutral',
 'Neutral'

In [32]:
#ratemd['Sentiment']=sentiment
ratemd['Whole_Sentiment']=sentiment

In [122]:
df = df[df.Sentiment != "Neutral"]

In [123]:
df

,Doc_names,Review_phrases,Sentiment
1,Dr. A. Michael Drachler,He is simply the best,Positive
4,Dr. A. Michael Drachler,Drachler is very knowledgeable,Positive
5,Dr. A. Michael Drachler,His bedside manners are simply perfect,Positive
11,Dr. A. Michael Drachler,Drachler is an excellent example of how a phy...,Positive
18,Dr. A. Michael Drachler,'It was my first,Positive
22,Dr. A. Michael Drachler,He was rude,Negative
23,Dr. A. Michael Drachler,had the worst bedside manner,Negative
24,Dr. A. Michael Drachler,I felt very bad about this visit,Negative
30,Dr. A. Michael Drachler,Michael Drachler is God-sent directly from He...,Positive
35,Dr. A. Michael Drachler,He is so extremely caring,Negative


In [320]:
sent = []
for x in range(len(df["Sentiment"])):
    if df["Sentiment"][x] > 0:
        sent.append("Positive")
    elif df["Sentiment"][x] < 0:
        sent.append("Negative")
    else:
        sent.append("Neutral")
    

In [250]:
len(sent)

6621

In [321]:
df["sent"] = sent

In [126]:
df_negative.to_csv(r'/Users/sujaikarunakaran/Desktop/Assignments/Spring 2020/Ranga/Final Project/Final/RateMds/Phrases_Negative.csv', header=True, index=False)


In [127]:
df_positive.to_csv(r'/Users/sujaikarunakaran/Desktop/Assignments/Spring 2020/Ranga/Final Project/Final/RateMds/Phrases_Positive.csv', header=True, index=False)


In [33]:
ratemd.to_csv(r'/Users/sujaikarunakaran/Desktop/Assignments/Spring 2020/Ranga/Final Project/Final/Healthgrades/All_Healthgrades_v3 copy1.csv', header=True, index=False)
